In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
import matplotlib.pyplot as plt
import cv2
import os
IM_HEIGHT = IM_WIDTH = 224

In [2]:
resnet = ResNet50(include_top=False, weights="imagenet", pooling='avg')
model = Sequential()
model.add(resnet)
model.add(Dropout(0.2))
model.add(Dense(1))
# model.layers[0].trainable = True
# sgd = SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
# model.compile(loss='kld', optimizer=sgd, metrics=['accuracy'])

2022-09-13 08:37:06.282853: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-09-13 08:37:06.283048: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1 Max


In [3]:
model.load_weights("../models/model-ms-all-resnet-no-adj.h5")

In [4]:
def format_im(im_path):
    im = cv2.imread(im_path)
    if im is None: return None
    im_size = im.shape[0]
    resized_im = cv2.resize(im, (224, 224))
    normed_im = np.array([(resized_im - 127.5) / 127.5])
    return im_size, normed_im

In [5]:
im_path = "../data/img_processed/face_data_cropped"
im_names = os.listdir(im_path)
im_names = list(filter(lambda x: x.endswith(".jpg") or x.endswith(".jpeg") or x.endswith(".png") or x.endswith(".webp"), im_names))
im_data = np.zeros((len(im_names), IM_HEIGHT, IM_WIDTH, 3))
im_sizes = np.zeros(len(im_names))

for i, im_name in enumerate(im_names):
    im_size, im = format_im(os.path.join(im_path, im_name))
    if im is not None:
        im_data[i] = im
        im_sizes[i] = im_size
    

In [6]:
scores = model.predict(im_data)

2022-09-13 08:37:15.879954: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-09-13 08:37:16.160746: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


16/16 [==============================] - 5s 143ms/step


In [9]:
df = pd.concat([pd.read_csv("../data/raw/data_1-200.csv"), pd.read_csv("../data/raw/data_200-482.csv")])
df = df.iloc[:-2, :-1]

In [41]:
sbject_id = np.array([name[:name.find(".")] for name in im_names]).astype("float").astype("str")

In [42]:
score_data = pd.DataFrame({"subjectID": object_id, "beauty_score": scores[:,0], "im_size": im_sizes})

In [20]:
df = df.merge(score_data, on="Code")

In [21]:
df.to_csv("../data/processed/all_data-no-adj.csv")

In [12]:
score_data.sort_values(by="beauty_score")

,Code,beauty_score
127,380.0,-0.900636
327,25.0,-0.500675
168,12.0,-0.090172
467,389.0,-0.047260
150,466.0,0.050845
...,...,...
398,146.0,4.460413
419,384.0,4.504300
315,317.0,4.531922
239,18.0,4.668997


In [8]:
def get_images(im_path, id_dtype="float"):
    im_names = os.listdir(im_path)
    im_names = list(filter(lambda x: x.endswith(".jpg") or x.endswith(".jpeg") or x.endswith(".png") or x.endswith(".webp"), im_names))
    im_data = np.zeros((len(im_names), IM_HEIGHT, IM_WIDTH, 3))
    im_sizes = np.zeros(len(im_names))
    im_id = np.array([name[:name.find(".")] for name in im_names]).astype(id_dtype)

    for i, im_name in enumerate(im_names):
        im_size, im = format_im(os.path.join(im_path, im_name))
        if im is not None:
            im_data[i] = im
            im_sizes[i] = im_size
    return im_id, im_sizes, im_data

def predict_scores(im_data, model):
    scores = model.predict(im_data)
    return scores

In [10]:
df2 = pd.read_csv("../data/raw/data-2.csv")

In [11]:
df2.head()

,ID,Agency,Name,Sector,Position,Remuneration,Expenses,Sequence#,Photo
0,1,Powerex Corporation,"Dobson Mack, Gp",Crown Corportaions & Others,Inter Utility Coord & Planning Mgr,"$187,086.00","$5,849.00",268,Y
1,2,University of British Columbia,Johnston Richard G C,Universities & Colleges,Professor (Tenure),"$187,015.00","$7,109.00",268,Y
2,3,University of British Columbia,Keown Paul Anthony,Universities & Colleges,Professor (Grant Tenure),"$187,005.00",$0.00,268,Y
3,4,University of British Columbia,Leyton-Brown Kevin,Universities & Colleges,Professor (Tenure),"$186,995.00","$23,120.00",268,Y
4,5,BC Hydro and Power Authority,"Zarin, Jn",Crown Corportaions & Others,Project Manager 3,"$186,985.68","$4,895.07",268,N


In [12]:
im_path = "../data/img_processed/face_data-2_cropped"
im_id, im_sizes, im_data = get_images(im_path, id_dtype="int64")
scores = predict_scores(im_data, model)
score_data = pd.DataFrame({"ID": im_id, "beauty_score": scores[:,0], "im_size": im_sizes})
df2 = df2.merge(score_data, on="ID")
df2

2/2 [==============================] - 0s 409ms/step


,ID,Agency,Name,Sector,Position,Remuneration,Expenses,Sequence#,Photo,beauty_score,im_size
0,1,Powerex Corporation,"Dobson Mack, Gp",Crown Corportaions & Others,Inter Utility Coord & Planning Mgr,"$187,086.00","$5,849.00",268,Y,2.450746,228.0
1,2,University of British Columbia,Johnston Richard G C,Universities & Colleges,Professor (Tenure),"$187,015.00","$7,109.00",268,Y,2.587546,196.0
2,3,University of British Columbia,Keown Paul Anthony,Universities & Colleges,Professor (Grant Tenure),"$187,005.00",$0.00,268,Y,2.948578,281.0
3,4,University of British Columbia,Leyton-Brown Kevin,Universities & Colleges,Professor (Tenure),"$186,995.00","$23,120.00",268,Y,3.717003,502.0
4,6,Delta Police,"Pike, Lorne",Police Departments,Superintendent,"$186,975.00","$3,303.00",268,Y,4.074679,289.0
5,8,Provincial government,"Read, Amy",Government,Crown Counsel Level 3,"$186,967.00",$0.00,268,Y,3.282667,130.0
6,10,Provincial government,"Adams, Alisia K.",Government,Crown Counsel Level 3,"$186,959.00",$0.00,268,Y,0.510718,1128.0
7,15,Provincial government,"Bergeron, Caroline",Government,Legal Counsel,"$173,770.00",$0.00,365,Y,3.894206,105.0
8,17,Provincial government,"Warburton, Rolf",Government,Legal Counsel,"$173,762.00",$0.00,365,Y,3.049936,699.0
9,18,Simon Fraser University,"Zhang, Richard",Universities & Colleges,Professor,"$173,746.00","$18,802.00",365,Y,2.874556,293.0


In [13]:
df2.to_csv("../data/processed/all_data-2-no-adj.csv")

In [14]:
score_data.to_csv("../data/processed/score_data-2-no-adj.csv")

In [69]:
df['Remuneration'].str.replace('[?\n $]|[,g]', '', regex=True)

0      395556.00
1      494122.00
2      389343.00
3      374025.00
4      370113.00
         ...    
476    120259.00
477    120217.00
478    120217.00
479    119955.00
480    118543.00
Name: Remuneration, Length: 481, dtype: object

In [72]:
df['Remuneration'].str.replace('[?\n$]', '', regex=True) 

0      395,556.00 
1       494,122.00
2      389,343.00 
3      374,025.00 
4      370,113.00 
          ...     
476    120,259.00 
477    120,217.00 
478    120,217.00 
479    119,955.00 
480    118,543.00 
Name: Remuneration, Length: 481, dtype: object